This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2026.1_tutorials/wallaroo-model-operations-tutorials/deploy/by-framework/xgboost).

## XGBoost Booster Binary Classification Deployment Tutorial

The following tutorial demonstrates deploying and serving an XGBoost Booster Binary Classification model to Wallaroo.

The following XGBoost model types are supported by Wallaroo.  XGBoost models **not** supported by Wallaroo are supported via the [Custom Models, also known as Bring Your Own Predict (BYOP)](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/).

| XGBoost Model Type | Wallaroo Auto Packaging Supported |
|---|---|
| XGBClassifier | √ |
| XGBRegressor | √ |
| Booster Classifier | √ |
| Booster Classifier | √ |
| Booster Regressor | √ |
| Booster Random Forest Regressor | √ |
| Booster Random Forest Classifier | √ |
| XGBRFClassifier | √ |
| XGBRFRegressor | √ |
| XGBRanker\* | X |

* XGBRanker XGBoost models are currently supported via converting them to [BYOP](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/) models.

## Goal

Upload, deploy, and serve a sample XGBoost Booster Binary Classification model.

### Resources

This tutorial provides the following:

* Models:
  * `/models/xgb_booster_binary_classification.pkl`: The sample XGBoost model that receives the `sklearn.datasets.load_breast_cancer` dataset.

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-configure/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-configure/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
* A X64 Docker deployment to deploy the model on an edge location.
* The notebook "Wallaroo Run Anywhere Model Drift Observability with Assays: Preparation" has been run, and the model edge deployments executed.

## Steps

### Import Libraries

The first step is to import the libraries we will need.  See `./requirements.txt` for a list of additional libraries used with this tutorial.

In [1]:
import wallaroo
from wallaroo.deployment_config import DeploymentConfigBuilder
import pyarrow as pa
from wallaroo.framework import Framework

import pickle
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from xgboost import train, DMatrix

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more details on logging in through Wallaroo, see the [Wallaroo SDK Essentials Guide: Client Connection](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client()

### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.



In [3]:
workspace_name = f'xgboost-booster-binary-classification'
pipeline_name = f'xgboost-booster-binary-classification'

model_name = 'xgboost-booster-binary-classification'
model_file_name = './models/xgb_booster_binary_classification.pkl'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Upload XGBoost Model

XGBoost models are uploaded to Wallaroo through the `wallaroo.client.Client.upload_model` method.

#### Upload XGBoost Model Parameters

The following parameters are available for XGBoost models.  

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Required*) | Set as `Framework.XGBOOST`. |
|`input_schema` | `pyarrow.lib.Schema` (*Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

#### Upload XGBoost Model Return

The following is returned with a successful model upload and conversion.

| Field | Type | Description |
|---|---|---|
| `name` | string | The name of the model. |
| `version` | string | The model version as a unique UUID. |
| `file_name` | string | The file name of the model as stored in Wallaroo. |
| `image_path` | string | The image used to deploy the model in the Wallaroo engine. |
| `last_update_time` | DateTime | When the model was last updated. |#

### Configure Input and Output Schemas

First we configure the input and output schemas in PyArrow format.

In [5]:
input_schema = pa.schema([
    pa.field('inputs', pa.list_(pa.float32(), list_size=30))
])

output_schema = pa.schema([
    pa.field('probabilities', pa.float32()),
])

### Upload Model Example

With the input and output schemas defined, we now upload the XGBoost model.

In [6]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.XGBOOST, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.
Model is attempting loading to a native runtime.successful

Ready


Name,xgboost-booster-binary-classification
Version,e0cc05c4-e7b6-4283-b546-67370dbb9c65
File Name,xgb_booster_binary_classification.pkl
SHA,be6d8c51395196da4720951b45660840fecdd680e7d99003a966023873322d8b
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-19-Jul 16:35:00
Workspace id,41
Workspace name,xgboost-booster-binary-classification


### Deploy Pipeline

With the model uploaded and packaged, we add the model as a pipeline model step.  For our deployment, we will set a minimum deployment configuration - this is the amount of resources the deployed pipeline uses from the cluster.

Once set, we deploy the pipeline, which allocates the assigned resources for the cluster and makes it available for inference requests.

In [7]:
pipeline.add_model_step(model)

deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [8]:
pipeline.deploy(deployment_config=deployment_config)

name,xgboost-booster-binary-classification
created,2024-07-19 16:31:54.670581+00:00
last_updated,2024-07-19 16:35:03.975598+00:00
deployed,True
workspace_id,41
workspace_name,xgboost-booster-binary-classification
arch,x86
accel,none
tags,
versions,"f32cb23e-d3dd-4935-bbb2-50918b66be5d, 423fc80e-39d5-4442-bd83-4f3447a573d4, fc500b0c-6f64-48fa-b771-8b441cb3f919, 619d7e1a-1259-4c6c-b61a-7306da01139f"
steps,xgboost-booster-binary-classification


### Run Sample Inference

The dataset is from the `sklearn.datasets.load_breast_cancer` examples.  These are converted to a pandas DataFrame, that is submitted to the deployed model in Wallaroo for an inference request.

In [9]:
dataset = load_breast_cancer()
X, y = dataset.data, dataset.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
dtrain = DMatrix(X_train, label=y_train)
dtest = DMatrix(X_test, label=y_test)

data = dtest.get_data().todense()[:100]

In [10]:
import pandas as pd

dataframe = pd.DataFrame({"inputs": data.tolist()})
dataframe

,inputs
0,"[12.470000267028809, 18.600000381469727, 81.08..."
1,"[18.940000534057617, 21.309999465942383, 123.5..."
2,"[15.460000038146973, 19.479999542236328, 101.6..."
3,"[12.399999618530273, 17.68000030517578, 81.470..."
4,"[11.539999961853027, 14.4399995803833, 74.6500..."
...,...
95,"[9.777000427246094, 16.989999771118164, 62.5, ..."
96,"[20.1299991607666, 28.25, 131.1999969482422, 1..."
97,"[13.8100004196167, 23.75, 91.55999755859375, 5..."
98,"[11.149999618530273, 13.079999923706055, 70.87..."


In [11]:
results = pipeline.infer(dataframe)
results

,time,in.inputs,out.probabilities,anomaly.count
0,2024-07-19 16:35:06.728,"[12.470000267, 18.6000003815, 81.0899963379, 4...",0.894763,0
1,2024-07-19 16:35:06.728,"[18.9400005341, 21.3099994659, 123.5999984741,...",0.107060,0
2,2024-07-19 16:35:06.728,"[15.4600000381, 19.4799995422, 101.6999969482,...",0.107060,0
3,2024-07-19 16:35:06.728,"[12.3999996185, 17.6800003052, 81.4700012207, ...",0.894763,0
4,2024-07-19 16:35:06.728,"[11.5399999619, 14.4399995804, 74.6500015259, ...",0.894763,0
...,...,...,...,...
95,2024-07-19 16:35:06.728,"[9.7770004272, 16.9899997711, 62.5, 290.200012...",0.894763,0
96,2024-07-19 16:35:06.728,"[20.1299991608, 28.25, 131.1999969482, 1261.0,...",0.107060,0
97,2024-07-19 16:35:06.728,"[13.8100004196, 23.75, 91.5599975586, 597.7999...",0.107060,0
98,2024-07-19 16:35:06.728,"[11.1499996185, 13.0799999237, 70.8700027466, ...",0.894763,0


### Undeploy the Pipeline

With the tutorial complete, we undeploy the pipeline and return the resources back to the cluster.

In [12]:
pipeline.undeploy()

name,xgboost-booster-binary-classification
created,2024-07-19 16:31:54.670581+00:00
last_updated,2024-07-19 16:35:03.975598+00:00
deployed,False
workspace_id,41
workspace_name,xgboost-booster-binary-classification
arch,x86
accel,none
tags,
versions,"f32cb23e-d3dd-4935-bbb2-50918b66be5d, 423fc80e-39d5-4442-bd83-4f3447a573d4, fc500b0c-6f64-48fa-b771-8b441cb3f919, 619d7e1a-1259-4c6c-b61a-7306da01139f"
steps,xgboost-booster-binary-classification
